<a href="https://colab.research.google.com/github/gdeslo/GEDI_course/blob/notebook/GEDI_OZ_L2B_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with GEDI L2B Version 2 Data in Python
### This tutorial demonstrates how to work with the Canopy Cover and Vertical Profile Metrics ([GEDI02_B.002](https://doi.org/10.5067/GEDI/GEDI02_B.002)) data product.
The Global Ecosystem Dynamics Investigation ([GEDI](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/gedi-overview/)) mission aims to characterize ecosystem structure and dynamics to enable radically improved quantification and understanding of the Earth's carbon cycle and biodiversity. The GEDI instrument produces high resolution laser ranging observations of the 3-dimensional structure of the Earth. GEDI is attached to the International Space Station and collects data globally between 51.6 N and 51.6 S latitudes at the highest resolution and densest sampling of any light detection and ranging (lidar) instrument in orbit to date. The Land Processes Distributed Active Archive Center (LP DAAC) distributes the GEDI Level 1 and Level 2 Version 1 and Version 2 products. The L1B and L2 GEDI products are archived and distributed in the HDF-EOS5 file format.

---
## Use Case Example:  
This tutorial was developed using one of Q-ForestLab's current projects as an example. The QPRP project covers a network of permanent tropical rainforest plots in North-Eastern Queensland, Australia that spans both a rainfall and altitudinal gradient.  **The goal of this practical is to use GEDI L2B Version 2 data to observe tree canopy height, cover, and profile over the QPRP network.**

This tutorial will show how to use Python to open GEDI L2B Version 2 files, visualize the sub-orbit of GEDI points (shots), subset to a region of interest, visualize GEDI canopy height and vertical profile metrics, and export subsets of GEDI science dataset (SDS) layers as GeoJSON files that can be loaded into GIS and/or Remote Sensing software programs.

***    
### Data Used in the Example:  
- **GEDI L2B Canopy Cover and Vertical Profile Metrics Data Global Footprint Level - [GEDI02_B.002](https://doi.org/10.5067/GEDI/GEDI02_B.002)**  
     - _The purpose of the L2B dataset is to extract biophysical metrics from each GEDI waveform. These metrics are based on the directional gap probability profile derived from the L1B waveform and include canopy cover, Plant Area Index (PAI), Plant Area Volume Density (PAVD) and Foliage Height Diversity (FHD)._   
     - **Science Dataset (SDS) layers:**
        - /geolocation/digital_elevation_model
        - /geolocation/elev_lowestmode  
        - /geolocation/elev_highestreturn  
        - /geolocation/lat_lowestmode  
        - /geolocation/lon_lowestmode  
        - /rh100  
        - /l2b_quality_flag  
        - /degrade_flag  
        - /sensitivity  
        - /pai  
        - /pavd_z  
        - /geolocation/shot_number  
        - /dz  
        - /selected_l2a_algorithm
***  
# Topics Covered:
1. [**Get Started**](#getstarted)  
    1.1 Install packages\
    1.2 Import Packages    
    1.3 Set Up the Working Environment and Retrieve Files      
2. [**Import and Interpret Data**](#importinterpret)      
    2.1 Open a GEDI HDF5 File and Read File Metadata     
    2.2 Read SDS Metadata and Subset by Beam   
3. [**Visualize a GEDI Sub-Orbit**](#visualizeorbit)      
    3.1 Subset by Layer and Create a Geodataframe   
    3.2 Visualize a Geodataframe
4. [**Work with GEDI L2B Data**](#L2B)        
    4.1 Import and Extract PAVD   
    4.2 Visualize PAVD    
5. [**Work with GEDI L2B Beam Transects**](#beamtransects)        
    5.1 Quality Filtering        
    5.2 Plot Beam Transects  
    5.3 Subset Beam Transects  
6. [**Plot Profile Transects**](#plottransects)      
    6.1 Plot PAVD Transects  
7. [**Spatial Visualization**](#spatialvisualization)      
    7.1 Import, Subset, and Quality Filter all Beams  
    7.2 Spatial Subsetting  
    7.3 Visualize All Beams: Canopy Height, Elevation, and PAI  
8. [**Export Subsets as GeoJSON Files**](#exportgeojson)     
       
***
# Before Starting this Tutorial:

This tutorial requires a compatible Python Environment and GEDI L2B granule from June 19, 2019 (orbit 02932, sub-orbit `02`) to download. To setup the Python environment and download the file, follow the steps in the github.

## Source Code used to Generate this Tutorial:
The repository containing all of the required files is located at: https://github.com/gdeslo/GEDI_course.git
  
<div class="alert alert-block alert-warning" >
<b>NOTE:</b> This tutorial was developed for GEDI L2B Version 2 HDF-EOS5 files and should only be used for that product. </div>   


---
# 1. Get Started <a id="getstarted"></a>

## 1.1 Install Packages <a id="1.1"></a>
#### Install the required packages.

In [ ]:
!pip install geospatial geoviews
!pip install "ipywidgets>=7,<8"

## 1.2 Import Packages <a id="1.1"></a>
#### Import the required packages and set the input/working directory to run this Google Colab Notebook locally.

In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
import geopandas as gp
from shapely.geometry import Point
import geoviews as gv
from geoviews import opts, tile_sources as gvts
import holoviews as hv
gv.extension('bokeh', 'matplotlib')
import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

os.chdir('../../')

## 1.3 Set Up the Working Environment and Retrieve Files<a id="1.2"></a>
#### The input directory is defined as the current working directory. Note that you will need to have the Google Colab Notebook and example data (.h5 and .geojshpson) stored in this directory in order to execute the tutorial successfully.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change the directory to your specific working folder
new_dir = os.path.join('/content/drive/MyDrive', 'Colab Notebooks', 'GEDI_course')
os.chdir(new_dir)
# Verify the current directory
print(os.getcwd())

You will need to download the file in order to execute this tutorial. Make sure to download the file into the `DATA` directory you made when setting up Google Colab.
You can use `earthaccess` package to download the data.

In [ ]:
# !pip install earthaccess==0.8.2
import earthaccess
earthaccess.login(persist=True)
results = ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/GEDI02_B.002/GEDI02_B_2022344195102_O22626_01_T06507_02_003_01_V002/GEDI02_B_2022344195102_O22626_01_T06507_02_003_01_V002.h5']
# download
downloaded_files = earthaccess.download(
    results,
    local_path='DATA/',
)

In [ ]:
gediFiles = [g for g in os.listdir('DATA/') if g.startswith('GEDI02_B') and g.endswith('.h5')]  # List all GEDI L2B .h5 files in inDir
gediFiles

---
# 2. Import and Interpret Data <a id="importinterpret"></a>

## 2.1 Open a GEDI HDF5 File and Read File Metadata <a id="2.1"></a>
#### Read the file using `h5py`. This package is particularly useful for storing large amounts of numerical data.

In [ ]:
L2B = f'DATA/{gediFiles[0]}'
L2B

#### The standard format  for GEDI Version 2 filenames is as follows:
> **GEDI02_B**: Product Short Name    
**2019170155833**: Julian Date and Time of Acquisition (YYYYDDDHHMMSS)  
**O02932**: Orbit Number   
**02**: Sub-Orbit Granule Number (1-4)  
**T02267**: Track Number (Reference Ground Track)   
**02**: Positioning and Pointing Determination System (PPDS) type (00 is predict, 01 rapid, 02 and higher is final)  
**003**: PGE Version Number    
**01**: Granule Production Version  
**V002**: Product Version  

Read in a GEDI HDF5 file using the `h5py` package.

In [ ]:
gediL2B = h5py.File(L2B, 'r')  # Read file using h5py

Navigate the HDF5 file below.

In [ ]:
list(gediL2B.keys())

The GEDI HDF5 file contains groups in which data and metadata are stored.
First, the `METADATA` group contains the file-level metadata.

In [ ]:
list(gediL2B['METADATA'])

This contains useful information such as the creation date, PGEVersion, and VersionID. Below, print the file-level metadata attributes.

In [ ]:
for g in gediL2B['METADATA']['DatasetIdentification'].attrs:
    print(g)

In [ ]:
print(gediL2B['METADATA']['DatasetIdentification'].attrs['purpose'])

## 2.2 Read SDS Metadata and Subset by Beam <a id="2.2"></a>

The GEDI instrument consists of 3 lasers producing a total of 8 beam ground transects. The eight remaining groups contain data for each of the eight GEDI beam transects. For additional information, be sure to check out: https://gedi.umd.edu/instrument/specifications/.

In [ ]:
beamNames = [g for g in gediL2B.keys() if g.startswith('BEAM')]
beamNames

One useful piece of metadata to retrieve from each beam transect is whether it is a full power beam or a coverage beam.
> The **Full Power Beam** involves operating the lidar at its maximum power output to achieve the highest possible signal-to-noise ratio. This can be particularly useful for penetrating dense canopies or for obtaining high-quality data in challenging conditions.

> The **Coverage Beam** is focused on maximizing the spatial coverage of the lidar measurements. It might involve using a wider beam or a different scanning pattern to cover a larger area.

In [ ]:
for g in gediL2B['BEAM0000'].attrs: print(g)

In [ ]:
for b in beamNames:
    print(f"{b} is a {gediL2B[b].attrs['description']}")

Below, pick one of the full power beams that will be used to retrieve GEDI L2B shots in Section 3.

In [ ]:
beamNames = ['BEAM1000']

Identify all the objects in the GEDI HDF5 file below.

In [ ]:
gediL2B_objs = []
gediL2B.visit(gediL2B_objs.append)                                           # Retrieve list of datasets
gediSDS = [o for o in gediL2B_objs if isinstance(gediL2B[o], h5py.Dataset)]  # Search for relevant SDS inside data file
[i for i in gediSDS if beamNames[0] in i][:10]                               # Print the first 10 datasets for selected beam

---
# 3. Visualize a GEDI Orbit <a id="visualizeorbit"></a>
#### In the section below, import GEDI L2B SDS layers into a `GeoPandas` GeoDataFrame for the beam specified above.
#### Use the `lat_lowestmode` and `lon_lowestmode` to create a `shapely` point for each GEDI shot location.

## 3.1 Subset by Layer and Create a Geodataframe <a id="3.1"></a>

Read in the SDS and take a representative sample (every 100th shot) and append to lists, then use the lists to generate a `pandas` dataframe.

In [ ]:
lonSample, latSample, shotSample, qualitySample, beamSample = [], [], [], [], []  # Set up lists to store data

# Open the SDS
lats = gediL2B[f'{beamNames[0]}/geolocation/lat_lowestmode'][()]
lons = gediL2B[f'{beamNames[0]}/geolocation/lon_lowestmode'][()]
shots = gediL2B[f'{beamNames[0]}/geolocation/shot_number'][()]
quality = gediL2B[f'{beamNames[0]}/l2b_quality_flag'][()]

# Take every 100th shot and append to list
for i in range(len(shots)):
    if i % 100 == 0:
        shotSample.append(str(shots[i]))
        lonSample.append(lons[i])
        latSample.append(lats[i])
        qualitySample.append(quality[i])
        beamSample.append(beamNames[0])

# Write all of the sample shots to a dataframe
latslons = pd.DataFrame({'Beam': beamSample, 'Shot Number': shotSample, 'Longitude': lonSample, 'Latitude': latSample,
                         'Quality Flag': qualitySample})
latslons

Above is a dataframe containing columns describing the beam, shot number, lat/lon location, and quality information about each shot.

Below, create an additional column called 'geometry' that contains a `shapely` point generated from each lat/lon location from the shot.

In [ ]:
# Take the lat/lon dataframe and convert each lat/lon to a shapely point
latslons['geometry'] = latslons.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)

Next, convert to a `Geopandas` GeoDataFrame.

In [ ]:
# Convert to a Geodataframe
latslons = gp.GeoDataFrame(latslons)
latslons = latslons.drop(columns=['Latitude','Longitude'])
latslons['geometry']

Pull out and plot an example `shapely` point below.

In [ ]:
latslons['geometry'][0]

## 3.2 Visualize a GeoDataFrame <a id="3.2"></a>
#### In this section, use the GeoDataFrame of Queensland and the QPRP plots, and use the `leafmap` python package to spatially visualize the location of the GEDI shots on a basemap.

Import a ShapeFile of Queensland and the QPRP plots as a GeoDataFrame. Note that you will need to have downloaded the folder with DATA to your Google Drive.

In [ ]:
import geopandas as gpd
plots = 'DATA/plots.shp'
plots_gdf = gpd.read_file(plots, encoding='ISO-8859-1')
QLD = gpd.read_file('DATA/QPRP-plots.geojson')

Define the center of Queensland and it's coordinates.

In [ ]:
center = QLD.geometry.centroid
#center = shapely.centroid(QLD)
center_coords = [center.iloc[0].y, center.iloc[0].x]
#center_coords = [center['geometry'][0].y, center['geometry'][0].x]
center_coords

Now we make a simple interactive map using `leafmap` that shows Queensland and the QPRP plots. A new window might open on the right; you can simply close it and continue running the script.

In [ ]:
import leafmap
m = leafmap.Map(center = center_coords, zoom = 5) #Make a map with the center set as the center of Queensland and zoomed in on this point
m.add_basemap('HYBRID') #Set a basemap as background
m.add_gdf(QLD, layer_name="Queensland", style={'fillColor': 'yellow', 'color': 'yellow', 'fillOpacity': 0.5}) #Add the shape of Queensland to the map
m.add_markers(plots_gdf, shape="circle", radius=4, color="blue", fill_color="blue", popup=["plot_name"]) #Add the QPRP plots to the map as markers
m #display the map

Congrats, you made your first map!

Next, let's make a new map that also adds all the GEDI shots from the chosen beam.

In [ ]:
m = leafmap.Map(center = center_coords, zoom = 7) #Make a new map with the same settings as the previous one
m.add_basemap('HYBRID')
m.add_gdf(QLD, layer_name="Queensland", style={'fillColor': 'yellow', 'color': 'yellow', 'fillOpacity': 0.5})
m.add_markers(plots_gdf, shape="circle", radius=4, color="blue", fill_color="blue", popup=["plot_name"])
m.add_markers(latslons, shape="circle", radius=2, color="red", fill_color="red", popup=["geometry", "Beam", "Shot Number", "Quality Flag"]) #This time we also add the shots from the selected GEDI beam
m

Above is a good illustration of the new GEDI _Version 2_ sub-orbit granules (remember that GEDI _Version 1_  files are stored as one ISS orbit). One of the benefits of using 	`leafmap` is the interactive nature of the output maps. Use the tools in the upper right of the map and try zooming in to find interesting shots close to QPRP plots.

Side Note: Wondering what the 0's and 1's for `l2b_quality_flag` mean?

In [ ]:
print(f"Quality Flag: {gediL2B[b]['l2b_quality_flag'].attrs['description']}")

Above, 0 is poor quality and a quality_flag value of 1 indicates the laser shot meets criteria based on energy, sensitivity, amplitude, and real-time surface tracking quality. We will show an example of how to quality filter GEDI data in section 5.1.
After finding one of the shots close to a QPRP plot and with a quality index of 1, find the index for that shot number so that we can find the correct shot to visualize in Section 4.

#### Each GEDI shot has a unique shot identifier (shot number) that is available within each data group of the product. The shot number is important to retain in any data subsetting as it will allow the user to link any shot record back to the original orbit data, and to link any shot and its data between the L1 and L2 products. The standard format  for GEDI Shots is as follows:

### Shot: 29320600200465601
> **2932**: Orbit Number      
**06**: Beam Number    
**0**: Reserved for future use     
**02**: Sub-orbit Granule Number    
**004**: Minor frame number   
**65601**: Shot index  

### 🧐 Question to Think About:

- How does the spatial distribution of GEDI shots relate to the ISS orbit path? Why do they appear in a narrow strip?



---
# 4. Work with GEDI L2B Data <a id="L2B"></a>
#### The L2B product contains biophysical information derived from the geolocated GEDI return waveforms including total and vertical profiles of canopy cover and Plant Area Index (PAI), the vertical Plant Area Volume Density (PAVD) profile, and Foliage Height Diversity (FHD).
Detailed product information can be found on the [GEDI L2B Product Page](https://doi.org/10.5067/GEDI/GEDI02_B.002).

## 4.1 Import and Extract Specific Shots
Notice that there are over a thousand datasets available in the GEDI L2B product. In the code blocks below, you will subset to just a few of the datasets available.

This section teaches how to extract and subset specific shots and plot Plant Area Volume Density (PAVD) using `matplotlib`.
`matplotlib` is a comprehensive library for creating static, animated, and interactive visualizations in Python,
making it a valuable tool for data analysis and visualization.

In [ ]:
len(gediSDS)

In [ ]:
beamNames

In [ ]:
beamSDS = [g for g in gediSDS if beamNames[0] in g]  # Subset to a single beam
len(beamSDS)

We will set 2 shot indices as an example. These shotindices will work for all the following code. Feel free to set it to different shots, however take into account that it is not guaranteed that everything will work.

In [ ]:
shot1 = 226260800100074501
shot2 = 226260800100074101

In [ ]:
index1 = np.where(gediL2B[f'{beamNames[0]}/shot_number'][()]==shot1)[0][0]  # Set the index for the first shot identified above
index1

In [ ]:
index2 = np.where(gediL2B[f'{beamNames[0]}/shot_number'][()]==shot2)[0][0]  # Set the index for the second shot identified above
index2

## 4.2 Visualize PAVD
Now we import the PAVD metrics (`pavd_z`) and begin exploring how to plot them.

In [ ]:
pavd = gediL2B[[g for g in beamSDS if g.endswith('/pavd_z')][0]]  # PAVD

Print the description for the PAVD dataset.

In [ ]:
print(f"Plant Area Volume Density is {pavd.attrs['description']}")

Below, open the `dz` layer in order to define the correct vertical step size.

In [ ]:
# Grab vertical step size
dz = gediL2B[f'{beamNames[0]}/ancillary/dz'][0]
dz

So the vertical step size is 5.0 meters.

In [ ]:
print(f"The shape of PAVD is {pavd.shape}.")

And it looks like PAVD includes 30 "steps" in each shot, describing the PAVD at height = step # * `dz`.

Now, bring in other useful L2B datasets such as `elev_lowestmode`, `lat_lowestmode` and `lon_lowestmode`.

In [ ]:
# Bring in the desired SDS
elev = gediL2B[f'{beamNames[0]}/geolocation/elev_lowestmode'][()]  # Latitude
lats = gediL2B[f'{beamNames[0]}/geolocation/lat_lowestmode'][()]  # Latitude
lons = gediL2B[f'{beamNames[0]}/geolocation/lon_lowestmode'][()]  # Longitude

Grab the location, elevation, and PAVD metrics for the shots defined above:

In [ ]:
shotElev1 = elev[index1]
shotLat1 = lats[index1]
shotLon1 = lons[index1]
shotPAVD1 = pavd[index1]

#now for shot 2
shotElev2 = elev[index2]
shotLat2 = lats[index2]
shotLon2 = lons[index2]
shotPAVD2 = pavd[index2]

Put everything together to identify the shots that we want to extract:

In [ ]:
print(f"The first shot is located at: {str(shotLat1)}, {str(shotLon1)} (shot ID: {shot1}, index {index1}) and is from {beamNames[0]}.")
print(f"The second shot is located at: {str(shotLat2)}, {str(shotLon2)} (shot ID: {shot2}, index {index2}) and is from {beamNames[0]}.")

Next, reformat PAVD into a list of uplets containing each PAVD value and height.

In [ ]:
# Append the first shot to the list
pavdAll1 = []
pavdElev1 = []

for i, e in enumerate(range(len(shotPAVD1))):
    if shotPAVD1[i] > 0:
        pavdElev1.append((shot1, shotElev1 + dz * i, shotPAVD1[i]))  # Append tuple of shot number, elevation, and PAVD
pavdAll1.append(pavdElev1)                                         # Append to final list

# Append the second shot to the list
pavdAll2 = []
pavdElev2 = []

for i, e in enumerate(range(len(shotPAVD2))):
    if shotPAVD2[i] > 0:
        pavdElev2.append((shot2, shotElev2 + dz * i, shotPAVD2[i]))  # Append tuple of shot number, elevation, and PAVD
pavdAll2.append(pavdElev2)                                         # Append to final list

Below, plot each shot by using `HoloView`. This high-level visualization library makes it easy to generate interactive plots with minimal code.

In [ ]:
pavdAll1

In [ ]:
pavdAll2

In [ ]:
# Create Holoviews Paths
# Be careful to change the shot numbers if necessary
path1 = hv.Path(pavdAll1, vdims='PAVD').opts(
    color='PAVD', clim=(0, 0.1), cmap='Greens', line_width=20, colorbar=True,
    width=700, height=550, clabel='PAVD', xlabel='Shot Number - 226260800100074501',
    ylabel='Elevation (m)', fontsize={'title': 16, 'xlabel': 16, 'ylabel': 16,
                                      'xticks': 12, 'yticks': 12,
                                      'clabel': 12, 'cticks': 10},
    shared_axes=False  # ✅ Allows independent Y-axis for visibility
)

path2 = hv.Path(pavdAll2, vdims='PAVD').opts(
    color='PAVD', clim=(0, 0.1), cmap='Greens', line_width=20, colorbar=True,
    width=700, height=550, clabel='PAVD', xlabel='Shot Number - 226260800100074101',
    ylabel='Elevation (m)', fontsize={'title': 16, 'xlabel': 16, 'ylabel': 16,
                                      'xticks': 12, 'yticks': 12,
                                      'clabel': 12, 'cticks': 10},
    shared_axes=False  # ✅ Different scaling for better visibility
)

# Display side by side without shared axes
(path1 + path2).opts(shared_axes=False)



### 🎉 Congratulations! You have plotted your first PAVD profile.

Now, let's analyze the results and reflect on the data.



### 🧐 Questions to Think About:
   - What do you notice about the axes for elevation? How does this affect the way PAVD is displayed?
   - Why do the two plots have different elevation ranges? What does this tell us about the two shots?
   - If we had used the same y-axis limits for both graphs, what would change?
   - Which regions in the plot have the highest PAVD values? What does this mean in terms of canopy structure?
   - Is the PAVD distribution continuous or patchy? Why might this be the case?
   - How does the PAVD vary with elevation? Is there a pattern you can identify?
   - The PAVD is color-coded using a gradient from light green to dark green. What does a darker color indicate?
   - Does the color scale influence how you interpret the data? If so, how?
   - If we used a different colormap (e.g., ‘Reds’ instead of ‘Greens’), how might that affect interpretation?
   - What could cause one shot to have higher PAVD values at certain elevations than the other?
   - How do these profiles relate to forest structure? Can you infer anything about canopy height or density?
   - If you were collecting more data, what other variables would you want to include for a better understanding of PAVD?

**Extra**: Using what you've learned, describe what you would expect to see in a PAVD profile for:
   - A dense tropical rainforest
   - An open savanna
   - A recently logged forest

---
# 5. Work with GEDI L2B Beam Transects<a id="beamtransects"></a>
#### Next, import a number of desired SDS layers for BEAM0110 (for the entire orbit) and create a `pandas` Dataframe to store the arrays.

In [ ]:
# Open all of the desired SDS
dem = gediL2B[[g for g in beamSDS if g.endswith('/digital_elevation_model')][0]][()]
zElevation = gediL2B[[g for g in beamSDS if g.endswith('/elev_lowestmode')][0]][()]
zHigh = gediL2B[[g for g in beamSDS if g.endswith('/elev_highestreturn')][0]][()]
zLat = gediL2B[[g for g in beamSDS if g.endswith('/lat_lowestmode')][0]][()]
zLon = gediL2B[[g for g in beamSDS if g.endswith('/lon_lowestmode')][0]][()]
canopyHeight = gediL2B[[g for g in beamSDS if g.endswith('/rh100')][0]][()]
quality = gediL2B[[g for g in beamSDS if g.endswith('/l2b_quality_flag')][0]][()]
degrade = gediL2B[[g for g in beamSDS if g.endswith('/degrade_flag')][0]][()]
sensitivity = gediL2B[[g for g in beamSDS if g.endswith('/sensitivity')][0]][()]
pavd = gediL2B[f'{beamNames[0]}/pavd_z'][()]
shotNums = gediL2B[f'{beamNames[0]}/shot_number'][()]
selectedAlgorithmL2A = gediL2B[[g for g in beamSDS if g.endswith('/selected_l2a_algorithm')][0]][()]

# Create a shot index
shotIndex = np.arange(shotNums.size)

In the GEDI L2B product, Canopy Height is stored in units (cm), so below convert to meters.

In [ ]:
canopyHeight = canopyHeight / 100  # Convert RH100 from cm to m

As mentioned in the sections above, Plant Area Volume Density (pavd) is defined as the _Vertical Plant Area Volume Density profile with a vertical step size of dZ_. Below, reformat the shape of the PAVD layer in order to add it to the dataframe below.

In [ ]:
print(f"The shape of Canopy Height is {canopyHeight.shape} vs. the shape of PAVD, which is {pavd.shape}.")

Above, notice that unlike a SDS layer like Canopy Height, which has a single value for each shot, PAVD has 30 values (representing different vertical heights) for each shot.

Below, reformat the data into a list of values for each shot:

In [ ]:
# Set up an empty list to append to
pavdA = []
for i in range(len(pavd)):

    # If any of the values are fill value, set to nan
    pavdF = [np.nan]
    for p in range(len(pavd[i])):
        if pavd[i][p]!= -9999:
            pavdF.append(pavd[i][p])  # If the value is not fill value, append to list
    pavdA.append(pavdF)               # Append back to master list

Below, notice the reformatted PAVD layer, which should now fit into the dataframe created below.

In [ ]:
len(pavdA)

In [ ]:
# Take the DEM, GEDI-produced Elevation, and Canopy height and add to a Pandas dataframe
transectDF = pd.DataFrame({
    'Shot Index': shotIndex,
    'Shot Number': shotNums,
    'Latitude': zLat,
    'Longitude': zLon,
    'Tandem-X DEM': dem,
    'Elevation (m)': zElevation,
    'Canopy Elevation (m)': zHigh,
    'Canopy Height (rh100)': canopyHeight,
    'Quality Flag': quality,
    'Degrade Flag': degrade,
    'Plant Area Volume Density': pavdA,
    'Sensitivity': sensitivity,
    'Selected L2A Algorithm': selectedAlgorithmL2A
    })

In [ ]:
transectDF

In [ ]:
index1

Notice the unusual values listed above--those shots are flagged as poor quality and will be removed in Section 5.1.


Now that you have the desired SDS into a `pandas` dataframe, begin plotting the entire beam transect:

In [ ]:
# Plot Canopy Height
canopyVis = hv.Scatter((transectDF['Shot Index'], transectDF['Canopy Height (rh100)']))
canopyVis.opts(color='darkgreen', height=500, width=900, title=f'GEDI L2B Full Transect {beamNames[0]}',
               fontsize={'title':16, 'xlabel':16, 'ylabel': 16}, size=0.1, xlabel='Shot Index', ylabel='Canopy Height (m)')

#### Congratulations! You have plotted your first **GEDI sub-orbit beam transect**. Notice above that things look a little messy--before we dive deeper into plotting full transects, let's quality filter the shots in the section below.

### 🧐 Questions to Think About:

- Do you notice any patterns in the canopy height across the transect?  
- What could be the cause of the sharp peaks and valleys in the graph?  
- Are there sections of the graph where canopy height is more consistent? What might this indicate about the underlying terrain or vegetation?  
- The plot appears "messy" with scattered points. What might be causing this?  
- Could some of these low canopy height values be from non-vegetated surfaces (e.g., roads, water bodies)?  
- What kinds of errors or uncertainties might be present in GEDI measurements? How can filtering (e.g., using the quality flag) help?  
- If you plotted a different GEDI beam, would you expect to see similar patterns, or would they be different? Why?  
- How might the same transect look in a dense tropical forest versus an open savanna?  
- If you had access to ground-based LiDAR data, how could you validate the accuracy of these canopy height measurements?  


## 5.1 Quality Filtering
#### Now that you have the desired layers imported as a dataframe for the entire beam transect, let's perform quality filtering.
#### Below, remove any shots where the `l2b_quality_flag` is set to 0 by defining those shots as `nan`.
#### The syntax of the line below can be read as: in the dataframe, find the rows "where" the quality flag is not equal (ne) to 0. If a row (shot) does not meet the condition, set all values equal to `nan` for that row.

In [ ]:
transectDF = transectDF.where(transectDF['Quality Flag'].ne(0))  # Set any poor quality returns to NaN

In [ ]:
transectDF

Below, quality filter even further by using the `degrade_flag` (Greater than zero if the shot occurs during a degrade period, zero otherwise) and the `Sensitivity` layer, using a threshold of 0.95.

In [ ]:
transectDF = transectDF.where(transectDF['Degrade Flag'] < 1)
transectDF = transectDF.where(transectDF['Sensitivity'] > 0.95)

Below, drop all of the shots that did not pass the quality filtering standards outlined above from the `transectDF`.

In [ ]:
transectDF = transectDF.dropna()  # Drop all of the rows (shots) that did not pass the quality filtering above

In [ ]:
print(f"Quality filtering complete, {len(transectDF)} high quality shots remaining.")

## 5.2 Plot Beam Transects
#### Next, plot the full remaining transect of high quality values using `holoviews` Scatter(). Combine the Tandem-X derived elevation, the GEDI-derived elevation, and the Canopy Top Elevation in a combined holoviews plot.

In [ ]:
# Plot Digital Elevation Model
demVis = hv.Scatter((transectDF['Shot Index'], transectDF['Tandem-X DEM']), label='Tandem-X DEM')
demVis = demVis.opts(color='black', height=500, width=900, fontsize={'xlabel': 16, 'ylabel': 16}, size=2)

In [ ]:
# Plot GEDI-Retrieved Elevation
zVis = hv.Scatter((transectDF['Shot Index'], transectDF['Elevation (m)']), label='GEDI-derived Elevation')
zVis = zVis.opts(color='saddlebrown', height=500, width=900, fontsize={'xlabel': 16, 'ylabel': 16}, size=2)

In [ ]:
# Plot Canopy Top Elevation
rhVis = hv.Scatter((transectDF['Shot Index'], transectDF['Canopy Elevation (m)']), label='Canopy Top Elevation')
rhVis = rhVis.opts(color='darkgreen', height=500, width=900, fontsize={'xlabel': 16, 'ylabel': 16}, size=2)


In [ ]:
# Combine all three scatterplots with improved axis limits
combined_plot = (demVis * zVis * rhVis).opts(
    show_legend=True,
    legend_position='top_left',
    fontsize={'title': 14, 'xlabel': 16, 'ylabel': 16},
    title=f'{beamNames[0]} Full Transect: {L2B.split("_")[0]}',
    ylim=(0, 1350),  # Set elevation range
    xlim=(transectDF['Shot Index'].min()*0.5, transectDF['Shot Index'].max()*1.1),  # Adjust x-axis to remove excessive gaps
    xticks=10,  # Adjust number of x-axis ticks for better readability
    yticks=10,  # Adjust y-axis ticks
    tools=['hover'],  # Enable hover for better exploration
    responsive=True
)

combined_plot

#### The plot still looks a bit messy this far zoomed out--feel free to pan, zoom, and explore different areas of the plot.

#### 🧐 Questions to Think About:
- How does the **Tandem-X DEM** compare to the **GEDI-derived elevation**? What could explain any differences?  
- What does the **Canopy Top Elevation** represent, and how is it different from the other two variables?  
- Where in the transect do you see the highest canopy elevations? What could explain these peaks?  
- Do you notice areas where the Tandem-X DEM and GEDI-derived elevation diverge significantly? What might cause this discrepancy?  
- Are there sections where canopy height is consistently low or missing? What could this indicate about the landscape?  
- Looking at the x-axis, do you see any changes in terrain? Can you hypothesize what type of environment this transect might pass through (e.g., dense forest, open land, mountains)?  
- The plot is "messy" - what do you think causes this?  
- What role might terrain slope, dense vegetation, or surface water play in affecting these elevation measurements?  
- Could there be errors in the data? If so, how would you detect and correct them?  
- How might cloud cover or atmospheric conditions impact GEDI’s elevation and canopy height estimates?
- If you analyzed a different GEDI beam, how might this graph look different?  
- How do you expect elevation and canopy height patterns to change in a tropical rainforest compared to an open woodland?  
- If you were to analyze multiple transects, what trends would you look for to study forest structure on a larger scale?  


## 5.3 Subset Beam Transects

Now, subset down to a smaller transect centered on the shot analyzed in the sections above.

First let's check out the indeces:

In [ ]:
index1

In [ ]:
index2

Below, we display the dataset to verify the available shot indices and their structure.

In [ ]:
transectDF

We now extract a smaller transect around the selected shot indices.  
To ensure consistency in indexing, we first determine the closest valid row positions in the dataset.

In [ ]:
# Ensure index is sorted before searching
sorted_index = np.sort(transectDF.index.values)

# Find the closest valid index positions
index1_pos = np.searchsorted(sorted_index, index1, side="left")
index2_pos = np.searchsorted(sorted_index, index2, side="left")

# Ensure the positions remain within valid bounds
index1_pos = min(max(0, index1_pos), len(transectDF) - 1)
index2_pos = min(max(0, index2_pos), len(transectDF) - 1)

# Print positions for verification
print(f"index1_pos: {index1_pos}, index2_pos: {index2_pos}")

To extract a subset of the transect, we select 50 points before and 50 points after the central shot.

In [ ]:
# Define subset ranges
start1 = max(0, index1_pos - 50)
end1 = min(len(transectDF), index1_pos + 50)
transectDF1 = transectDF.iloc[start1:end1]

start2 = max(0, index2_pos - 50)
end2 = min(len(transectDF), index2_pos + 50)
transectDF2 = transectDF.iloc[start2:end2]

# Print subset lengths for validation
print(f"Length of transectDF1: {len(transectDF1)}")
print(f"Length of transectDF2: {len(transectDF2)}")


Below, we display the extracted transects to verify the selection.

In [ ]:
# Display the extracted transects
transectDF1, transectDF2


---
# 6. Plot Profile Transects <a id="plottransects"></a>
#### In this section, plot the transect subset using elevation, canopy height, and plant area volume density (PAVD) metrics.

In order to get an idea of the length of the beam transect that you are plotting, you can plot the x-axis as distance, which is calculated below.

In [ ]:
# Calculate along-track distance for each shot
distance1 = np.arange(0.0, len(transectDF1.index) * 60, 60)  # GEDI Shots are spaced 60 m apart
transectDF1['Distance'] = distance1                          # Add Distance as a new column in the dataframe
distance2 = np.arange(0.0, len(transectDF2.index) * 60, 60)  # GEDI Shots are spaced 60 m apart
transectDF2['Distance'] = distance2                          # Add Distance as a new column in the dataframe

## 6.1 Plot PAVD Transects

Similar to what was done with PAVD in the sections above, reformat PAVD into a list of tuples containing each PAVD value and height by shot.

In [ ]:
pavdAll1 = []
for j, s in enumerate(transectDF1.index):
    pavdShot1 = transectDF1['Plant Area Volume Density'][s]
    elevShot1 = transectDF1['Elevation (m)'][s]
    pavdElev1 = []

    # Remove fill values
    if np.isnan(pavdShot1).all():
        continue
    else:
        del pavdShot1[0]
    for i, e in enumerate(range(len(pavdShot1))):
        if pavdShot1[i] > 0:
            pavdElev1.append((distance1[j], elevShot1 + dz * i, pavdShot1[i]))  # Append tuple of distance, elevation, and PAVD
    pavdAll1.append(pavdElev1)                                                # Append to final list


pavdAll2 = []
for j, s in enumerate(transectDF2.index):
    pavdShot2 = transectDF2['Plant Area Volume Density'][s]
    elevShot2 = transectDF2['Elevation (m)'][s]
    pavdElev2 = []

    # Remove fill values
    if np.isnan(pavdShot2).all():
        continue
    else:
        del pavdShot2[0]
    for i, e in enumerate(range(len(pavdShot2))):
        if pavdShot2[i] > 0:
            pavdElev2.append((distance2[j], elevShot2 + dz * i, pavdShot2[i]))  # Append tuple of distance, elevation, and PAVD
    pavdAll2.append(pavdElev2)                                                # Append to final list

In [ ]:
pavdAll1

In [ ]:
pavdAll2

In [ ]:
canopyElevation1 = [p[-1][1] for p in pavdAll1 if p]  # Only use non-empty lists
canopyElevation2 = [p[-1][1] for p in pavdAll2 if p]


In [ ]:
print("pavdAll1 structure:", type(pavdAll1), "Length:", len(pavdAll1))
print("Example entry in pavdAll1:", pavdAll1[0] if pavdAll1 else "Empty")
print("pavdAll2 structure:", type(pavdAll2), "Length:", len(pavdAll2))
print("Example entry in pavdAll2:", pavdAll2[0] if pavdAll2 else "Empty")


Below, plot each shot by using `holoviews` Path() function, with the PAVD plotted in the third dimension in shades of green.

In [ ]:
# Remove empty sublists
pavdAll1 = [sublist for sublist in pavdAll1 if sublist]
pavdAll2 = [sublist for sublist in pavdAll2 if sublist]

# Convert to NumPy arrays for consistency
pavdAll1 = [np.array(sublist) for sublist in pavdAll1]
pavdAll2 = [np.array(sublist) for sublist in pavdAll2]


In [ ]:
# Be careful to change the shot numbers if necessary
path1 = hv.Path(pavdAll1, vdims='PAVD').options(color='PAVD', clim=(0,0.3), cmap='Greens', line_width=8, colorbar=True,
                                               width=950, height=500, ylim=(min(transectDF1['Elevation (m)']) - 5, max(canopyElevation1) + 5), clabel='PAVD', xlabel='Distance Along Transect (m) - 226260800100074501',
                                               ylabel='Elevation (m)', fontsize={'title':16, 'xlabel':16, 'ylabel': 16,
                                                                                 'xticks':12, 'yticks':12,
                                                                                 'clabel':12, 'cticks':10})

path2 = hv.Path(pavdAll2, vdims='PAVD').options(color='PAVD', clim=(0,0.3), cmap='Greens', line_width=8, colorbar=True,
                                                  width=950, height=500, ylim=(min(transectDF1['Elevation (m)']) - 5, max(canopyElevation1) + 5), clabel='PAVD', xlabel='Distance Along Transect (m) - 226260800100074101',
                                                  ylabel='Elevation (m)', fontsize={'title':16, 'xlabel':16, 'ylabel': 16,
                                                                                    'xticks':12, 'yticks':12,
                                                                                    'clabel':12, 'cticks':10})



# Display them separately instead of forcing alignment
hv.Layout([path1, path2]).opts(shared_axes=False)


Add in the ground elevation and canopy top elevation for better context as to where in the canopy the highest PAVD exists.

In [ ]:
path1_2 = hv.Curve((distance1, transectDF1['Elevation (m)']), label='Ground Elevation').options(color='black', line_width=2)
path2_2 = hv.Curve((distance2, transectDF2['Elevation (m)']), label='Ground Elevation').options(color='black', line_width=2)

path1_3 = hv.Curve((distance1, canopyElevation1), label='Canopy Top Elevation').options(color='grey', line_width=1.5)
path2_3 = hv.Curve((distance2, canopyElevation2), label='Canopy Top Elevation').options(color='grey', line_width=1.5)

In [ ]:
# Plot all three together
# Be careful to change the shot numbers if necessary
path1 = path1 * path1_2 * path1_3
path1.opts(height=500,width=980, ylim=(min(transectDF1['Elevation (m)']) - 5, max(canopyElevation1) + 5),
          xlabel='Distance Along Transect (m) - 226260800100074501', ylabel='Elevation (m)', legend_position='bottom_right',
          fontsize={'title':15, 'xlabel':15, 'ylabel': 15, 'xticks': 14, 'yticks': 14, 'legend': 14},
          title=f'GEDI L2B {beamNames[0]} PAVD over Queensland, Australia in 2022')

path2 = path2 * path2_2 * path2_3
path2.opts(height=500,width=980, ylim=(min(transectDF1['Elevation (m)']) - 5, max(canopyElevation1) + 5),
          xlabel='Distance Along Transect (m) - 226260800100074101', ylabel='Elevation (m)', legend_position='bottom_right',
          fontsize={'title':15, 'xlabel':15, 'ylabel': 15, 'xticks': 14, 'yticks': 14, 'legend': 14},
          title=f'GEDI L2B {beamNames[0]} PAVD over Queensland, Australia in 2022')

# Display them separately instead of forcing alignment
hv.Layout([path1, path2]).opts(shared_axes=False)

In [ ]:
#save graph
hv.save(path1, 'DATA/GEDI_L2B_PAVD_1.html')
hv.save(path1, 'DATA/GEDI_L2B_PAVD_2.html')

Above, you can get an idea about the terrain over the region of interest. In terms of vegetation structure, this plot does a good job of showing not only which portions of the canopy are taller, but also where they are denser (darker shades of green).

#### 🧐 Questions to Think About:

- What do the different lines and color intensities represent in this plot?
- How does setting the same y-axis for both plots improve the comparison between the two transects?
- What are the key differences between the left and right plots? What might be causing these differences?
- How does the difference between ground elevation and canopy top elevation inform us about the vegetation structure in a given area?
- Where do you observe the largest difference between these two lines? What might that indicate?
- How does the topography influence canopy height? Do you see patterns where forested areas follow terrain contours?
- How does the distribution of PAVD align with canopy height and terrain changes?
- Are there any regions where PAVD is high, but canopy height is low? What might cause this?
- In which areas do you see more variation in PAVD, and how does this relate to different forest types?
- What landscape or ecological factors might explain the differences observed between these two transects?
- How might these results differ in different ecosystems, such as a tropical rainforest versus a savanna?
- How can this type of visualization be used in forest monitoring and conservation planning?
- How could changes in these plots over time help detect forest degradation, regrowth, or land-use changes?




At this point you have visualized the elevation, canopy, and vertical structure of specific footprints over the QPRP plots in Queensland, Australia and for a transect cutting through the region. In section 7 you will look at mapping all of the high-quality shots from all eight GEDI beams for a given region of interest in order to gain knowledge on the spatial distribution of and characteristics of the canopy over the QPRP plots.

---
# 7. Spatial Visualization<a id="spatialvisualization"></a>
#### Section 7 combines many of the techniques learned above including how to import GEDI datasets, perform quality filtering, spatial subsetting, and visualization.

## 7.1 Import, Subset, and Quality Filter All Beams

Below, re-open the GEDI L2B observation--but this time, loop through and import data for all 8 of the GEDI beams.

In [ ]:
beamNames = [g for g in gediL2B.keys() if g.startswith('BEAM')]

In [ ]:
beamNames

Loop through each of the desired datasets (SDS) for each beam, append to lists, and transform into a `pandas` DataFrame.

In [ ]:
# Set up lists to store data
shotNum, dem, zElevation, zHigh, zLat, zLon, canopyHeight, quality, degrade, sensitivity, pai, beamI, selectedAlgorithmL2A = ([] for i in range(13))

In [ ]:
# Loop through each beam and open the SDS needed
for b in beamNames:
    [shotNum.append(h) for h in gediL2B[[g for g in gediSDS if g.endswith('/shot_number') and b in g][0]][()]]
    [dem.append(h) for h in gediL2B[[g for g in gediSDS if g.endswith('/digital_elevation_model') and b in g][0]][()]]
    [zElevation.append(h) for h in gediL2B[[g for g in gediSDS if g.endswith('/elev_lowestmode') and b in g][0]][()]]
    [zHigh.append(h) for h in gediL2B[[g for g in gediSDS if g.endswith('/elev_highestreturn') and b in g][0]][()]]
    [zLat.append(h) for h in gediL2B[[g for g in gediSDS if g.endswith('/lat_lowestmode') and b in g][0]][()]]
    [zLon.append(h) for h in gediL2B[[g for g in gediSDS if g.endswith('/lon_lowestmode') and b in g][0]][()]]
    [canopyHeight.append(h) for h in gediL2B[[g for g in gediSDS if g.endswith('/rh100') and b in g][0]][()]]
    [quality.append(h) for h in gediL2B[[g for g in gediSDS if g.endswith('/l2b_quality_flag') and b in g][0]][()]]
    [degrade.append(h) for h in gediL2B[[g for g in gediSDS if g.endswith('/degrade_flag') and b in g][0]][()]]
    [sensitivity.append(h) for h in gediL2B[[g for g in gediSDS if g.endswith('/sensitivity') and b in g][0]][()]]
    [beamI.append(h) for h in [b] * len(gediL2B[[g for g in gediSDS if g.endswith('/shot_number') and b in g][0]][()])]
    [selectedAlgorithmL2A.append(h) for h in gediL2B[[g for g in gediSDS if g.endswith('/selected_l2a_algorithm') and b in g][0]][()]]
    [pai.append(h) for h in gediL2B[f'{b}/pai'][()]]

In [ ]:
# Convert lists to Pandas dataframe
allDF = pd.DataFrame({
    'Shot Number': shotNum,
    'Beam': beamI,
    'Latitude': zLat,
    'Longitude': zLon,
    'Tandem-X DEM': dem,
    'Elevation (m)': zElevation,
    'Canopy Elevation (m)': zHigh,
    'Canopy Height (rh100)': canopyHeight,
    'Quality Flag': quality,
    'Plant Area Index': pai,
    'Degrade Flag': degrade,
    'Sensitivity': sensitivity,
    'Selected L2A Algorithm': selectedAlgorithmL2A
    })

## 7.2 Spatial Subsetting
#### Below, subset the pandas dataframe using a simple bounding box region of interest. If you are interested in spatially clipping GEDI shots to a GeoJSON region of interest, be sure to check out the GEDI-Subsetter python script available at: https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-subsetter/browse.

In [ ]:
len(allDF)

More than 800,000 shots are contained in this single GEDI sub-orbit granule! Below subset down to only the shots falling within this small bounding box encompassing Queensland. `QLD_STATE_POLYGON` our `geopandas` geodataframe can be called for the "envelope" or smallest bounding box encompassing the entire region of interest. Here, use that as the bounding box for subsetting the GEDI shots.

In [ ]:
QLD.total_bounds

In [ ]:
minLon, minLat, maxLon, maxLat = QLD.total_bounds  # Define the min/max lat/lon from the bounds of Queensland

In [ ]:
minLon, minLat, maxLon, maxLat

Filter by the bounding box, which is done similarly to filtering by quality in section 6.1 above.

In [ ]:
allDF = allDF.where(allDF['Latitude'] > minLat)
allDF = allDF.where(allDF['Latitude'] < maxLat)
allDF = allDF.where(allDF['Longitude'] > minLon)
allDF = allDF.where(allDF['Longitude'] < maxLon)

In [ ]:
allDF = allDF.dropna()  # Drop shots outside of the ROI

In [ ]:
len(allDF)

Notice you have drastically reduced the number of shots you are working with (which will greatly enhance your experience in plotting them below). But first, remove any poor quality shots that exist within the ROI.

In [ ]:
# Set any poor quality returns to NaN
allDF = allDF.where(allDF['Quality Flag'].ne(0))
allDF = allDF.where(allDF['Degrade Flag'] < 1)
allDF = allDF.where(allDF['Sensitivity'] > 0.95)
allDF = allDF.dropna()
len(allDF)

Down to roughly 10000 shots, next create a `Shapely` Point out of each shot and insert it as the geometry column in the [soon to be geo]dataframe.

In [ ]:
# Take the lat/lon dataframe and convert each lat/lon to a shapely point
allDF['geometry'] = allDF.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)

In [ ]:
# Convert to geodataframe
allDF = gp.GeoDataFrame(allDF)
allDF = allDF.drop(columns=['Latitude','Longitude'])

## 7.3 Visualize All Beams: Canopy Height, Elevation, and PAI

Now, let's plot the `geopandas` GeoDataFrame using `leafmap`.

In [ ]:
m = leafmap.Map(center = center_coords, zoom = 7)
m.add_basemap('HYBRID')
m.add_gdf(QLD, layer_name="QPRP-CSIRO", style={'fillColor': 'yellow', 'color': 'yellow', 'fillOpacity': 0.5})
m.add_markers(allDF, shape="circle", radius=2, color="red", fill_color="red", popup=["geometry", "Beam", "Shot Number", "Quality Flag"])
m.add_markers(plots_gdf, shape="circle", radius=4, color="blue", fill_color="blue", popup=["plot_name"])
m

#### Feel free to pan and zoom in to the GEDI shots in yellow.

Now let's not only plot the points in the geodataframe but also add a colormap for Canopy Height (m), Elevation (m), and Plant Area Index (PAI).

In [ ]:
allDF['Canopy Height (rh100)'] = allDF['Canopy Height (rh100)'] / 100 # Convert canopy height from cm to m
#Watch out: do not run this cell twice! As the allDF['Canopy Height (rh100)'] will be divided by 100 again.

In [ ]:
m = leafmap.Map(center = center_coords, zoom = 7)
m.add_basemap('HYBRID')
m.add_gdf(QLD, layer_name="QPRP-CSIRO", style={'fillColor': 'yellow', 'color': 'yellow', 'fillOpacity': 0.2})
m.add_markers(plots_gdf, shape="circle", radius=4, color="blue", fill_color="blue", popup=["plot_name"])
m.add_data(allDF, column="Canopy Height (rh100)", cmap="plasma", marker_radius=2, layer_name="Canopy Height (rh100)")
m

Next, take a look at the GEDI-derived elevation over the shots. Notice below that the colormap is changed to 'terrain'.

In [ ]:
m = leafmap.Map(center = center_coords, zoom = 7)
m.add_basemap('HYBRID')
m.add_gdf(QLD, layer_name="QPRP-CSIRO", style={'fillColor': 'yellow', 'color': 'yellow', 'fillOpacity': 0.2})
m.add_markers(plots_gdf, shape="circle", radius=4, color="blue", fill_color="blue", popup=["plot_name"])
m.add_data(allDF, column="Elevation (m)", cmap="terrain", marker_radius=2, layer_name="GEDI Elevation")
m

Last but certainly not least, `Plant Area Index`:

In [ ]:
m = leafmap.Map(center = center_coords, zoom = 7)
m.add_basemap('HYBRID')
m.add_gdf(QLD, layer_name="QPRP-CSIRO", style={'fillColor': 'yellow', 'color': 'yellow', 'fillOpacity': 0.2})
m.add_markers(plots_gdf, shape="circle", radius=4, color="blue", fill_color="blue", popup=["plot_name"])
m.add_data(allDF, column="Plant Area Index", cmap="Greens", marker_radius=2, layer_name="GEDI PAI over Queensland")
m

###🧐 Questions to Think About:

- Do you notice any correlations between canopy height and elevation? What landscapes show the highest canopy heights?
- What variations in Plant Area Index (PAI) do you see along the transect? How might this relate to forest density?
- Are there regions where canopy height is low, but PAI is high? What could explain this?
- How could these maps help in identifying areas of deforestation or forest degradation?
- How could this data be integrated into forest conservation or carbon storage assessments?
- What are some potential errors or uncertainties in the GEDI data? How might atmospheric conditions or terrain features affect accuracy?
- If you had access to higher-resolution airborne LiDAR, how might it improve upon GEDI’s spatial coverage and detail?



## Success!
You have now learned how to start working with GEDI L2B files in Python as well as some interesting strategies for visualizing those data in order to better understand your specific region of interest. Using this Jupyter Notebook as a workflow, you should now be able to switch to GEDI files over your specific region of interest and re-run the notebook. Good Luck!


# 8. Export Subsets as GeoJSON Files<a id="exportgeojson"></a>
#### In this section, export the GeoDataFrame as a `.geojson` file that can be easily opened in your favorite remote sensing and/or GIS software and will include an attribute table with all of the shots/values for each of the SDS layers in the dataframe.

In [ ]:
gediL2B.filename  # L2B Filename

In [ ]:
outName = gediL2B.filename.replace('.h5', '.json')  # Create an output file name using the input file name
outName

In [ ]:
allDF.to_file(outName, driver='GeoJSON')  # Export to GeoJSON


## This document was based of of notebooks provided by NASA LP DAAC
### Please find their contact info below:

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://lpdaac.usgs.gov/>  
  

## Contact info

This notebook was made by Luna Soenens and Geike De Sloover for the Advanced Remote Sensing course of Ghent University.\
If you have any questions please feel free to contact us. Ask your question via email, teams or make a quick appointment and come ask us at Campus Coupure.

Email:

luna.soenens@ugent.be\
geike.desloover@ugent.be


